.
.

# MANUAL COBRApy
## ÚLTIMA VERSIÓN 
------------------------------------------------------------------------------------------
.
.
.
#### UNIVERSIDAD AUTÓNOMA METROPOLITANA UNIDAD CUAJIMALPA
.
.
.
#### Mauricio Carrasco González
#### Dr. Roberto Olivares Hernández
.
.
.
.
.



## Introducción


## Instalación


Cobrapy es un paquete de Python que proporciona una interfaz simple para la reconstrucción y el análisis basados en restricciones metabólicas.

El paquete incluye interfaces simples orientadas a objetos para la construcción de modelos (incluida la lectura hacia/desde los formatos sbml, matlab y json) e implementa métodos COBRA de uso común, como análisis de balance de flujo, análisis de variabilidad de flujo y análisis de eliminación de genes.

COBRApy se puede instalar con cualquier instalación reciente de pip. A continuación se muestran las instrucciones para varios sistemas operativos:

Mac OS X o Linux----------------------------------------------

Le recomendamos que cree un entorno virtual Python. Instale COBRApy cuando un entorno esté activo ejecutando el siguiente comando en la terminal:

In [ ]:
pip install cobra

Microsoft Windows-----------------------------------

Puede usar el mismo comando de instalación usando pip o usando Anaconda, puede abrir un shell Anaconda e instalar COBRApy desde el canal de bioconda.

Si desea cargar modelos MATLAB, necesitará dependencias adicionales. Por favor instalar:


In [ ]:
pip install cobra[all]

## Primeros Pasos

Como se abordó anteriormente, Cobrapy es una librería de Python que proporciona una interfaz simple para la reconstrucción y el análisis basados en restricciones metabólicas. Antes de comenzar a trabajar con este paquete es importante tener en mente que Cobrapy trabaja orientado a objetos, que en este contexto serán los elementos que podremos manipular de nuestro modelo. 

Los objetos que podemos utilizar en Cobrapy son: modelos, reacciones, metabolitos y genes. Estos objetos o elementos interaccionarán entre sí mediante sus "atributos" para representar el estado biológico del sistema simulado.

En Cobrapy, como en otras librerías de python, es necesario de importar algunas dependencias que sean necesarias en el espacio de trabajo actual.
Sugerimos que para el buen funcionamiento de scripts usando Cobrapy al menos las siguientes dependencias sean llamadas al inicio del código:

In [ ]:
from __future__ import print_function #dependencia que evitará problemas de versión de sintaxis
import cobra                          #Se llama a la librería de Cobrapy
from cobra import Model, Reaction, Metabolite #Se llama a los objetos que se utilizarán más adelante
from cobra.io import read_sbml_model  #Se llama a la dependencia que permitirá leer modelos metabólicos en formato sbml

### Cargando un modelo

En Cobrapy podemos cargar un modelo metabólico distribuido en la web o construir un nuevo modelo desde cero. También podemos cargar modelos de prueba de Salmonella y de E.coli que vienen instalados con la librería, para esto escribimos: 

In [1]:
from __future__ import print_function

import cobra
import cobra.test

Para crear un modelo de prueba y cargarlo como objeto "modelo" escirbimos:

In [2]:
#"ecoli" y "salmonella" son argumentos también validos
model = cobra.test.create_test_model("textbook")

una vez cargado el modelo, podemos obtener una visualización general de los elementos que lo conforman:

(NOTA: usando Jupyter Notebook la información se despliega tan sencillo como escribir el siguiente comando, en uso de Python de forma iterada, es necesario usar la función print()).
    

In [3]:
model

Name,e_coli_core
Memory address,0x012742da8
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


El siguiente caso es cargar un modelo SBML en formato XML que tengamos guardado en la misma dirección del archivo script que estamos generando, para esto es necesario inicializar el código llamando las dependencias necesarias:

In [4]:
from __future__ import print_function
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model  #inicializar la función de lectura de modelos sbml

Los modelos SBML (Systems Biology Markup Language) son modelos de distribución en formato estándar XML que tienen soporte para modelos COBRA a partir de la versión 2 de la extensión FBC.

Para leer un modelo SBML y cargarlo como objeto "modelo" escirbimos la siguientr función teniendo como argumento el nombre del archivo del modelo (para este ejemplo usaremos el modelo a escala genómica de Pichia pastoris descargado de la web):



In [5]:
model = read_sbml_model('MODEL1507180023_url.xml')

In [6]:
model.name #Regresa información del nombre del modelo metabólico

'Chung2010 - Genome-scale metabolic network of Pichia pastoris (iPP668)'

In [7]:
model

Name,MODEL1507180023
Memory address,0x012742e80
Number of metabolites,1178
Number of reactions,1361
Number of groups,0
Objective expression,1.0*BIOMASS - 1.0*BIOMASS_reverse_69053
Compartments,"Cytoplasm, Mitochondria, Extracellular, Glyoxysome, Endoplasmic reticulum, Nucleus, Golgi, Vacuole"


## Construcción de modelo


In [ ]:
---